In [1]:
import numpy as np
import pandas as pd
import time
import ast
import sqlite3
from DeepCCS.model.encoders import SmilesToOneHotEncoder
from DeepCCS.model.splitter import SMILESsplitter

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import keras
from keras.optimizers import adam, rmsprop
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Input, Concatenate
from keras.layers import Conv1D, MaxPooling1D,  Activation, BatchNormalization, Flatten
from keras import backend as K

Using TensorFlow backend.


In [2]:
HMDB_SQLITE_FILE = "hmdb_metabolites.sql"

We use an sqlite version of the original HMDB.xml file. It's easier to parse and re-use. Parsing script is HMDB_sql_converter.py

In [3]:
sql_connection = sqlite3.connect(HMDB_SQLITE_FILE)
cursor = sql_connection.cursor()
SMILES = []
polar_surface_area = []
logS = []
refractivity = []
polarizability = []
logP_alogps = []
logP_chemaxon = []
query = "SELECT DISTINCT SMILES, polar_surface_area, logS, refractivity, polarizability, \
logP_ALOGPS, logP_ChemAxon \
FROM HMDB where SMILES is not null and \
polar_surface_area is not null and \
logS is not null and \
refractivity is not null and \
polarizability is not null and \
logP_ALOGPS is not null and \
logP_ChemAxon is not null;"
for i in cursor.execute(query):
    SMILES.append(i[0])
    polar_surface_area.append(float(i[1]))
    logS.append(float(i[2]))
    refractivity.append(float(i[3]))
    polarizability.append(float(i[4]))
    logP_alogps.append(float(i[5]))
    logP_chemaxon.append(float(i[6]))

We now filter the dataset to keep only SMILES with an appropriate length.

In [4]:
Y = [np.array(polar_surface_area), np.array(logS), np.array(refractivity), np.array(polarizability),
     np.array(logP_alogps), np.array(logP_chemaxon)]
X = np.array(SMILES)

# Filter SMILES by length
smiles_splitter_multi = SMILESsplitter()
lengths = [len(smiles_splitter_multi.split(x)) for x in X]
lengths_filter = np.array(lengths) <= 250

X = X[lengths_filter.astype(bool)]
Y = [target[lengths_filter.astype(bool)] for target in Y]

The objective is to measure the impact of learning the internal representation using a multi-output problem and to re-use the internal representation to predict CCS. To make sure the SMILES encoder will be compatible with both problems, we will train a SMILES encoder that is compatible with both datasets.

In [5]:
smiles_encoder_ccs = SmilesToOneHotEncoder()
smiles_encoder_ccs.load_encoder("SMILES_encoder.json")

smiles_encoder_multi = SmilesToOneHotEncoder()
smiles_encoder_multi.fit(X)

all_symbols = smiles_encoder_ccs.converter.keys() + smiles_encoder_multi.converter.keys()
#all_symbols = set(all_symbols)

smiles_encoder_multi = SmilesToOneHotEncoder()
for i, j in enumerate(set(all_symbols)):
    smiles_encoder_multi.converter[j] = i
smiles_encoder_multi._is_fit = True

print("There is {} symbols in the SMILES encoder".format(len(smiles_encoder_multi.converter)))

There is 58 symbols in the SMILES encoder


We now have a SMILES encoder, a bunch of SMILES and a series of properties to predict.
We have to seperate the data between a train, valid and test set.

In [6]:
# Split train - test
np.random.seed(432)
mask_train = np.zeros(len(X), dtype=int)
mask_train[:int(len(X) * 0.7)] = 1
np.random.shuffle(mask_train)
mask_test = 1 - mask_train
mask_train = mask_train.astype(bool)
mask_test = mask_test.astype(bool)

X_pooled = X[mask_train]
X_test = X[mask_test]

Y_pooled = [i[mask_train] for i in Y]
Y_test = [i[mask_test] for i in Y]

# Split train - valid
mask_train = np.zeros(len(X_pooled), dtype=int)
mask_train[:int(len(X_pooled) * 0.9)] = 1
np.random.shuffle(mask_train)
mask_valid = 1 - mask_train
mask_train = mask_train.astype(bool)
mask_valid = mask_valid.astype(bool)

X_train = X_pooled[mask_train]
X_valid = X_pooled[mask_valid]

Y_train = [i[mask_train] for i in Y_pooled]
Y_valid = [i[mask_valid] for i in Y_pooled]

In [7]:
# SMILES encoding
X_train_encoded = smiles_encoder_multi.transform(X_train)
X_valid_encoded = smiles_encoder_multi.transform(X_valid)
X_test_encoded = smiles_encoder_multi.transform(X_test)
smiles_encoder_multi.save_encoder("Smiles_encoder_HMDB.json")

In [8]:
# Network structure
smile_input_layer = Input(shape=(250, len(smiles_encoder_multi.converter)), name="smiles")
conv = Conv1D(64, kernel_size=4, activation='relu', kernel_initializer='normal')(smile_input_layer)

previous = conv
for i in range(6):
    conv = Conv1D(64, kernel_size=4, activation='relu', kernel_initializer='normal')(previous)
    if i == 5:
        pool = MaxPooling1D(pool_size=2, strides=2)(conv)
    else:
        pool = MaxPooling1D(pool_size=2, strides=1)(conv)
    previous = pool

flat = Flatten()(previous)

# polar_surface_area
previous = flat
for i in range(2):
    dense_layer = Dense(384, activation="relu", kernel_initializer='normal')(previous)
    previous = dense_layer
output_logp = Dense(1, activation="linear", name="polar_surface_area")(previous)

# logS
previous = flat
for i in range(2):
    dense_layer = Dense(384, activation="relu", kernel_initializer='normal')(previous)
    previous = dense_layer
output_logs = Dense(1, activation="linear", name="logs")(previous)

# refractivity

previous = flat
for i in range(2):
    dense_layer = Dense(384, activation="relu", kernel_initializer='normal')(previous)
    previous = dense_layer
output_refractivity = Dense(1, activation="linear", name="refractivity")(previous)

# polarizability
previous = flat
for i in range(2):
    dense_layer = Dense(384, activation="relu", kernel_initializer='normal')(previous)
    previous = dense_layer
output_polarizability = Dense(1, activation="linear", name="polarizability")(previous)

#logP_alogps
previous = flat
for i in range(2):
    dense_layer = Dense(384, activation="relu", kernel_initializer='normal')(previous)
    previous = dense_layer
output_logp_alogps = Dense(1, activation="linear", name="logp_alogps")(previous)

#Logp_Chemaxon
previous = flat
for i in range(2):
    dense_layer = Dense(384, activation="relu", kernel_initializer='normal')(previous)
    previous = dense_layer
output_logp_chemaxon = Dense(1, activation="linear", name="logp_chemaxon")(previous)

# optimizer and compile
opt = getattr(keras.optimizers, 'adam')
opt = opt()
model = Model(input=smile_input_layer,
              outputs=[output_logp, output_logs, output_refractivity, output_polarizability, 
                       output_logp_alogps, output_logp_chemaxon])
model.compile(optimizer=opt, loss='mean_squared_error')

model.summary()

/home/pplante/tensorflow-py2.7.13-v2/lib/python2.7/site-packages/ipykernel_launcher.py:64: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("sm...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
smiles (InputLayer)             (None, 250, 58)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 247, 64)      14912       smiles[0][0]                     
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 244, 64)      16448       conv1d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 243, 64)      0           conv1d_2[0][0]                   
__________________________________________________________________________________________________
conv1d_3 (

In [9]:
model_file = "Model_multioutput_2018-11-28-001.model"
model_checkpoint = ModelCheckpoint(model_file, save_best_only=True, save_weights_only=True)

In [10]:
model.fit(X_train_encoded, Y_train,
          epochs=150,
          batch_size=20,
          validation_data=(X_valid_encoded, Y_valid),
          verbose=True,
          callbacks=[model_checkpoint])

Train on 71232 samples, validate on 7915 samples
Epoch 1/150
71232/71232 [==============================] - 135s 2ms/step - loss: 1044.8396 - polar_surface_area_loss: 559.5253 - logs_loss: 0.9922 - refractivity_loss: 375.9304 - polarizability_loss: 96.2346 - logp_alogps_loss: 2.0455 - logp_chemaxon_loss: 10.1117 - val_loss: 478.4878 - val_polar_surface_area_loss: 408.8167 - val_logs_loss: 0.2515 - val_refractivity_loss: 47.0289 - val_polarizability_loss: 19.9160 - val_logp_alogps_loss: 0.5834 - val_logp_chemaxon_loss: 1.8912
Epoch 2/150
71232/71232 [==============================] - 126s 2ms/step - loss: 83.2026 - polar_surface_area_loss: 28.0373 - logs_loss: 0.1928 - refractivity_loss: 41.1766 - polarizability_loss: 11.9473 - logp_alogps_loss: 0.5006 - logp_chemaxon_loss: 1.3480 - val_loss: 26.4182 - val_polar_surface_area_loss: 9.6854 - val_logs_loss: 0.1684 - val_refractivity_loss: 10.9951 - val_polarizability_loss: 4.1823 - val_logp_alogps_loss: 0.4287 - val_logp_chemaxon_loss: 0.9

71232/71232 [==============================] - 126s 2ms/step - loss: 10.5421 - polar_surface_area_loss: 3.8705 - logs_loss: 0.0835 - refractivity_loss: 4.8087 - polarizability_loss: 1.3193 - logp_alogps_loss: 0.1601 - logp_chemaxon_loss: 0.3001 - val_loss: 5.1055 - val_polar_surface_area_loss: 1.8924 - val_logs_loss: 0.0881 - val_refractivity_loss: 2.0018 - val_polarizability_loss: 0.6308 - val_logp_alogps_loss: 0.1702 - val_logp_chemaxon_loss: 0.3222
Epoch 19/150
71232/71232 [==============================] - 126s 2ms/step - loss: 10.7764 - polar_surface_area_loss: 4.4820 - logs_loss: 0.0792 - refractivity_loss: 4.4061 - polarizability_loss: 1.3545 - logp_alogps_loss: 0.1520 - logp_chemaxon_loss: 0.3027 - val_loss: 38.3591 - val_polar_surface_area_loss: 2.8892 - val_logs_loss: 0.0834 - val_refractivity_loss: 30.0950 - val_polarizability_loss: 4.6481 - val_logp_alogps_loss: 0.1704 - val_logp_chemaxon_loss: 0.4731
Epoch 20/150
71232/71232 [==============================] - 126s 2ms/step

71232/71232 [==============================] - 126s 2ms/step - loss: 6.0063 - polar_surface_area_loss: 2.1292 - logs_loss: 0.0609 - refractivity_loss: 2.6930 - polarizability_loss: 0.8377 - logp_alogps_loss: 0.1049 - logp_chemaxon_loss: 0.1806 - val_loss: 5.7039 - val_polar_surface_area_loss: 1.9915 - val_logs_loss: 0.0731 - val_refractivity_loss: 2.3835 - val_polarizability_loss: 0.8323 - val_logp_alogps_loss: 0.1551 - val_logp_chemaxon_loss: 0.2683
Epoch 36/150
71232/71232 [==============================] - 126s 2ms/step - loss: 6.9051 - polar_surface_area_loss: 2.1542 - logs_loss: 0.0600 - refractivity_loss: 3.4387 - polarizability_loss: 0.9565 - logp_alogps_loss: 0.1069 - logp_chemaxon_loss: 0.1889 - val_loss: 3.8725 - val_polar_surface_area_loss: 1.3243 - val_logs_loss: 0.0743 - val_refractivity_loss: 1.5261 - val_polarizability_loss: 0.5552 - val_logp_alogps_loss: 0.1362 - val_logp_chemaxon_loss: 0.2565
Epoch 37/150
71232/71232 [==============================] - 126s 2ms/step - l

71232/71232 [==============================] - 126s 2ms/step - loss: 6.0586 - polar_surface_area_loss: 2.4683 - logs_loss: 0.0528 - refractivity_loss: 2.5376 - polarizability_loss: 0.7670 - logp_alogps_loss: 0.0893 - logp_chemaxon_loss: 0.1435 - val_loss: 12.0017 - val_polar_surface_area_loss: 4.5216 - val_logs_loss: 0.0671 - val_refractivity_loss: 6.3521 - val_polarizability_loss: 0.6676 - val_logp_alogps_loss: 0.1233 - val_logp_chemaxon_loss: 0.2700
Epoch 53/150
71232/71232 [==============================] - 126s 2ms/step - loss: 5.2489 - polar_surface_area_loss: 1.8307 - logs_loss: 0.0529 - refractivity_loss: 2.3907 - polarizability_loss: 0.7397 - logp_alogps_loss: 0.0897 - logp_chemaxon_loss: 0.1452 - val_loss: 4.4216 - val_polar_surface_area_loss: 1.2308 - val_logs_loss: 0.0705 - val_refractivity_loss: 2.1303 - val_polarizability_loss: 0.5974 - val_logp_alogps_loss: 0.1358 - val_logp_chemaxon_loss: 0.2568
Epoch 54/150
71232/71232 [==============================] - 126s 2ms/step - 

71232/71232 [==============================] - 126s 2ms/step - loss: 3.9220 - polar_surface_area_loss: 1.1662 - logs_loss: 0.0472 - refractivity_loss: 1.8939 - polarizability_loss: 0.6200 - logp_alogps_loss: 0.0777 - logp_chemaxon_loss: 0.1170 - val_loss: 4.3074 - val_polar_surface_area_loss: 1.1182 - val_logs_loss: 0.0660 - val_refractivity_loss: 2.0738 - val_polarizability_loss: 0.6568 - val_logp_alogps_loss: 0.1284 - val_logp_chemaxon_loss: 0.2642
Epoch 70/150
71232/71232 [==============================] - 126s 2ms/step - loss: 4.1209 - polar_surface_area_loss: 1.2408 - logs_loss: 0.0472 - refractivity_loss: 2.0156 - polarizability_loss: 0.6233 - logp_alogps_loss: 0.0794 - logp_chemaxon_loss: 0.1148 - val_loss: 8.0528 - val_polar_surface_area_loss: 1.6255 - val_logs_loss: 0.0680 - val_refractivity_loss: 4.6405 - val_polarizability_loss: 1.2831 - val_logp_alogps_loss: 0.1339 - val_logp_chemaxon_loss: 0.3018
Epoch 71/150
71232/71232 [==============================] - 126s 2ms/step - l

71232/71232 [==============================] - 126s 2ms/step - loss: 3.6416 - polar_surface_area_loss: 1.3375 - logs_loss: 0.0421 - refractivity_loss: 1.5536 - polarizability_loss: 0.5388 - logp_alogps_loss: 0.0677 - logp_chemaxon_loss: 0.1019 - val_loss: 3.9579 - val_polar_surface_area_loss: 1.1763 - val_logs_loss: 0.0732 - val_refractivity_loss: 1.7434 - val_polarizability_loss: 0.5616 - val_logp_alogps_loss: 0.1348 - val_logp_chemaxon_loss: 0.2686
Epoch 90/150
71232/71232 [==============================] - 126s 2ms/step - loss: 3.6426 - polar_surface_area_loss: 1.3154 - logs_loss: 0.0424 - refractivity_loss: 1.5735 - polarizability_loss: 0.5410 - logp_alogps_loss: 0.0691 - logp_chemaxon_loss: 0.1013 - val_loss: 3.5297 - val_polar_surface_area_loss: 0.9499 - val_logs_loss: 0.0694 - val_refractivity_loss: 1.5670 - val_polarizability_loss: 0.5231 - val_logp_alogps_loss: 0.1450 - val_logp_chemaxon_loss: 0.2753
Epoch 91/150
71232/71232 [==============================] - 126s 2ms/step - l

71232/71232 [==============================] - 126s 2ms/step - loss: 5.1221 - polar_surface_area_loss: 2.0362 - logs_loss: 0.0415 - refractivity_loss: 2.1957 - polarizability_loss: 0.6737 - logp_alogps_loss: 0.0674 - logp_chemaxon_loss: 0.1076 - val_loss: 5.3797 - val_polar_surface_area_loss: 2.3323 - val_logs_loss: 0.0768 - val_refractivity_loss: 1.9737 - val_polarizability_loss: 0.5888 - val_logp_alogps_loss: 0.1403 - val_logp_chemaxon_loss: 0.2678
Epoch 107/150
71232/71232 [==============================] - 126s 2ms/step - loss: 4.2202 - polar_surface_area_loss: 1.7396 - logs_loss: 0.0418 - refractivity_loss: 1.6926 - polarizability_loss: 0.5753 - logp_alogps_loss: 0.0678 - logp_chemaxon_loss: 0.1031 - val_loss: 4.4810 - val_polar_surface_area_loss: 1.5532 - val_logs_loss: 0.0718 - val_refractivity_loss: 1.8469 - val_polarizability_loss: 0.6008 - val_logp_alogps_loss: 0.1383 - val_logp_chemaxon_loss: 0.2700
Epoch 108/150
71232/71232 [==============================] - 126s 2ms/step -

71232/71232 [==============================] - 126s 2ms/step - loss: 3.3400 - polar_surface_area_loss: 1.3333 - logs_loss: 0.0363 - refractivity_loss: 1.3676 - polarizability_loss: 0.4612 - logp_alogps_loss: 0.0581 - logp_chemaxon_loss: 0.0834 - val_loss: 5.4022 - val_polar_surface_area_loss: 1.7255 - val_logs_loss: 0.0740 - val_refractivity_loss: 2.3515 - val_polarizability_loss: 0.8591 - val_logp_alogps_loss: 0.1331 - val_logp_chemaxon_loss: 0.2591
Epoch 124/150
71232/71232 [==============================] - 125s 2ms/step - loss: 2.9331 - polar_surface_area_loss: 0.9402 - logs_loss: 0.0355 - refractivity_loss: 1.3576 - polarizability_loss: 0.4636 - logp_alogps_loss: 0.0566 - logp_chemaxon_loss: 0.0796 - val_loss: 5.2476 - val_polar_surface_area_loss: 2.4667 - val_logs_loss: 0.0761 - val_refractivity_loss: 1.6869 - val_polarizability_loss: 0.6164 - val_logp_alogps_loss: 0.1407 - val_logp_chemaxon_loss: 0.2607
Epoch 125/150
71232/71232 [==============================] - 126s 2ms/step -

71232/71232 [==============================] - 126s 2ms/step - loss: 2.9743 - polar_surface_area_loss: 1.2297 - logs_loss: 0.0338 - refractivity_loss: 1.1657 - polarizability_loss: 0.4155 - logp_alogps_loss: 0.0540 - logp_chemaxon_loss: 0.0755 - val_loss: 8.0392 - val_polar_surface_area_loss: 4.8246 - val_logs_loss: 0.0731 - val_refractivity_loss: 2.1781 - val_polarizability_loss: 0.5512 - val_logp_alogps_loss: 0.1393 - val_logp_chemaxon_loss: 0.2729
Epoch 141/150
71232/71232 [==============================] - 125s 2ms/step - loss: 2.7967 - polar_surface_area_loss: 0.9961 - logs_loss: 0.0340 - refractivity_loss: 1.2005 - polarizability_loss: 0.4362 - logp_alogps_loss: 0.0552 - logp_chemaxon_loss: 0.0747 - val_loss: 3.9484 - val_polar_surface_area_loss: 1.1924 - val_logs_loss: 0.0734 - val_refractivity_loss: 1.7389 - val_polarizability_loss: 0.5327 - val_logp_alogps_loss: 0.1436 - val_logp_chemaxon_loss: 0.2674
Epoch 142/150
71232/71232 [==============================] - 126s 2ms/step -

In [11]:
model.load_weights(model_file)
y_pred = model.predict(X_test_encoded)

In [12]:
from sklearn.metrics import median_absolute_error, mean_absolute_error, r2_score, mean_squared_error
def relative_mean(Y_true, Y_pred):
    mean = np.mean((abs(Y_pred - Y_true) / Y_true) * 100)
    return mean


def relative_median(Y_true, Y_pred):
    med = np.median((abs(Y_pred - Y_true) / Y_true) * 100)
    return med

def percentile_95(Y_true, Y_pred):
    percentile = np.percentile((abs(Y_pred - Y_true) / Y_true) * 100, 95)
    return percentile

def percentile_90(Y_true, Y_pred):
    percentile = np.percentile((abs(Y_pred - Y_true) / Y_true) * 100, 90)
    return percentile

properties = ["polar_surface_area", "logS", "refractivity", "polarizability", "logP_alogps", "logP_chemaxon"]
for i, p in enumerate(properties):
    print(p)
    mse = mean_squared_error(y_true=Y_test[i], y_pred = y_pred[i])
    r2 = r2_score(y_true=Y_test[i], y_pred = y_pred[i])
    mean_abs_err = mean_absolute_error(y_true=Y_test[i], y_pred = y_pred[i])
    median_abs_err = median_absolute_error(y_true=Y_test[i], y_pred = y_pred[i])
    relative_mean_err = relative_mean(Y_test[i], y_pred[i].flatten())
    relative_median_err = relative_median(Y_test[i], y_pred[i].flatten())
    ninety = percentile_90(Y_test[i], y_pred[i].flatten())
    ninety_five = percentile_95(Y_test[i], y_pred[i].flatten())
    print("MSE: {}".format(mse))
    print("R2: {}".format(r2))
    print("median absolute: {}".format(median_abs_err))
    print("mean absolute: {}".format(mean_abs_err))
    print("median relative: {}".format(relative_median_err))
    print("mean relative: {}".format(relative_mean_err))
    print("90 %: {}".format(ninety))
    print("95 %: {}".format(ninety_five))
    print("------------------")

polar_surface_area
MSE: 1.49993596077
R2: 0.999790565072
median absolute: 0.207430648804
mean absolute: 0.405067120394
median relative: 0.196761021294
mean relative: inf
90 %: 0.924156953381
95 %: 1.81314147699
------------------
logS
MSE: 0.0626097674573
R2: 0.986465915664
median absolute: 0.0434478759766
mean absolute: 0.122753823549
median relative: -0.584262924771
mean relative: nan
90 %: -0.0964498007169
95 %: -0.0445922114798
------------------
refractivity
MSE: 1.30885941873
R2: 0.999892614339
median absolute: 0.228576660156
mean absolute: 0.514369718147
median relative: 0.0813555313757
mean relative: 0.550275511923
90 %: 1.22038056478
95 %: 2.158981787
------------------
polarizability
MSE: 0.540969660975
R2: 0.999774111363
median absolute: 0.28466796875
mean absolute: 0.449278126543
median relative: 0.249770966982
mean relative: 0.766640085671
90 %: 1.98516230429
95 %: 3.36727405202
------------------
logP_alogps
MSE: 0.129241339133
R2: 0.990206607682
median absolute: 0.056222

/home/pplante/tensorflow-py2.7.13-v2/lib/python2.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in divide
  This is separate from the ipykernel package so we can avoid doing imports until
/home/pplante/tensorflow-py2.7.13-v2/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in divide
  
/home/pplante/tensorflow-py2.7.13-v2/lib/python2.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in divide
  app.launch_new_instance()
/home/pplante/tensorflow-py2.7.13-v2/lib/python2.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in divide
  if sys.path[0] == '':
/home/pplante/tensorflow-py2.7.13-v2/lib/python2.7/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


In [13]:
#Save the dataset in order to easily reproduce/reload the experiements
import h5py as h5
with h5.File("Multi-output_datasets.h5", 'w') as fo:
    fo.create_dataset("train_x", data=X_train_encoded)
    fo.create_dataset("train_y", data=Y_train)
    fo.create_dataset("valid_x", data=X_valid_encoded)
    fo.create_dataset("valid_y", data=Y_valid)
    fo.create_dataset("test_x", data=X_test_encoded)
    fo.create_dataset("test_y", data=Y_test)

# Retrain model for CCS prediction
Now that the multi-output model can perform predictions for multiple properties, we will use its internal representation to perform CCS prediction. This will tell us if we have enough data to train the internal representation for CCS predictions using only the CCS data. Furthermore, it will show that CNN are appropriate for molecular descriptor predictions directly from SMILES.

In [14]:
#Read the data exactly the same way as it was done for the first experiment
from DeepCCS.utils import read_dataset, filter_data
from DeepCCS.model.encoders import AdductToOneHotEncoder

datafile = "../DATASETS.h5"
datasets_names = ["MetCCS_pos", "MetCCS_neg", "Agilent_pos", "Agilent_neg", "Waters_pos", "Waters_neg",
                 "PNL", "McLean", "CBM"] ## TODO: Change PNL to PNNL in the h5 datafile

datasets = [read_dataset(datafile, d_name) for d_name in datasets_names] # Read
datasets = [filter_data(d_set) for d_set in datasets] # Filter

np.random.seed(777)
save_test_sets_data = {}
pooled_set = []
test_sets = []
train_set = []
validation_set = []
test_sets_names = []
for i, dset in enumerate(datasets_names):
    if dset in ["MetCCS_pos", "MetCCS_neg"]:
        pooled_set.append([np.array(datasets[i]["SMILES"]),
                           np.array(datasets[i]["Adducts"]),
                           np.array(datasets[i]["CCS"])])
    elif dset in ["Agilent_pos", "Agilent_neg", "Waters_pos", "Waters_neg"]:
        test_sets.append([np.array(datasets[i]["SMILES"]),
                          np.array(datasets[i]["Adducts"]),
                          np.array(datasets[i]["CCS"])])
        test_sets_names.append(dset)
    elif dset in ["PNL", "McLean", "CBM"]:
        smiles = np.array(datasets[i]["SMILES"])
        ccs = np.array(datasets[i]["CCS"])
        adducts = np.array(datasets[i]["Adducts"])
        
        # We use binary masks to split the datasets between pooled and test
        mask_pooled = np.zeros(len(smiles), dtype=int)
        mask_pooled[:int(len(smiles) * 0.8)] = 1  # The remaining 20% goes in the test set.
        np.random.shuffle(mask_pooled)
        mask_test = 1 - mask_pooled
        mask_pooled = mask_pooled.astype(bool)
        mask_test = mask_test.astype(bool)
        
        pooled_set.append([smiles[mask_pooled], adducts[mask_pooled], ccs[mask_pooled]])
        test_sets.append([smiles[mask_test], adducts[mask_test], ccs[mask_test]])
        test_sets_names.append(dset)
# Split pooled between train (90%) and validation (10%)
smiles_pooled = np.concatenate([i[0] for i in pooled_set])
adducts_pooled = np.concatenate([i[1] for i in pooled_set])
ccs_pooled = np.concatenate([i[2] for i in pooled_set])

mask_train = np.zeros(len(smiles_pooled), dtype=int)
mask_train[:int(len(smiles_pooled) * 0.9)] = 1  # The remaining 10% goes in the validation set.
np.random.shuffle(mask_train)
mask_valid = 1 - mask_train
mask_train = mask_train.astype(bool)
mask_valid = mask_valid.astype(bool)

train_set = [smiles_pooled[mask_train], adducts_pooled[mask_train], ccs_pooled[mask_train]]
validation_set = [smiles_pooled[mask_valid], adducts_pooled[mask_valid], ccs_pooled[mask_valid]]

#We use the SMILES encoder that was used for encoding the HMDB database.
train_set[0] = smiles_encoder_multi.transform(train_set[0])
validation_set[0] = smiles_encoder_multi.transform(validation_set[0])

adducts_encoder = AdductToOneHotEncoder()
adducts_encoder.fit(np.concatenate([dset["Adducts"] for dset in datasets]))
train_set[1] = adducts_encoder.transform(train_set[1])
validation_set[1] = adducts_encoder.transform(validation_set[1])

In [15]:
# Keep only the input layer and the conv. + max. pooling layers + the flatten layer
del model.layers[15:]

# Set trainable property at false to lock weights
for l in model.layers:
    l.trainable = False

    
adduct_input_layer = Input(shape=(len(adducts_encoder.converter),), name="adduct")

previous = model.layers[-1].output

remix_layer = keras.layers.concatenate([previous, adduct_input_layer], axis=-1)
previous = remix_layer

# Insert new dense layers
for i in range(2):
    dense_layer = Dense(384, activation="relu", kernel_initializer='normal')(previous)
    previous = dense_layer

#Insert new ouput layer
output = Dense(1, activation="linear")(previous)


# Compile
opt = adam(lr=0.0001)
new_model = Model(inputs=[model.layers[0].input, adduct_input_layer], outputs=output)
new_model.compile(optimizer=opt, loss='mean_squared_error')

new_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
smiles (InputLayer)             (None, 250, 58)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 247, 64)      14912       smiles[0][0]                     
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 244, 64)      16448       conv1d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 243, 64)      0           conv1d_2[0][0]                   
__________________________________________________________________________________________________
conv1d_3 (

In [16]:
model_file = "Model_CCS_after_Multioutput_20181128.h5"
model_checkpoint = ModelCheckpoint(model_file, save_best_only=True, save_weights_only=True)

In [17]:
#Convert the pure keras model to as DeepCCS model (done manually)
from DeepCCS.model.DeepCCS import DeepCCSModel

deepCCS_model = DeepCCSModel()
deepCCS_model.adduct_encoder = adducts_encoder
deepCCS_model.smiles_encoder = smiles_encoder_multi
deepCCS_model.model = new_model

In [18]:
deepCCS_model.train_model(X1_train= train_set[0], X2_train=train_set[1], Y_train=train_set[2],
                X1_valid=validation_set[0], X2_valid=validation_set[1], Y_valid=validation_set[2],
                model_checkpoint=model_checkpoint, nbr_epochs=150, verbose=1)

Train on 1665 samples, validate on 186 samples
Epoch 1/150
1665/1665 [==============================] - 8s 5ms/step - loss: 5484.5892 - val_loss: 175.4669
Epoch 2/150
1665/1665 [==============================] - 7s 4ms/step - loss: 166.7360 - val_loss: 143.9959
Epoch 3/150
1665/1665 [==============================] - 7s 4ms/step - loss: 134.4268 - val_loss: 118.9343
Epoch 4/150
1665/1665 [==============================] - 7s 4ms/step - loss: 114.1581 - val_loss: 81.8228
Epoch 5/150
1665/1665 [==============================] - 7s 4ms/step - loss: 101.7865 - val_loss: 64.3730
Epoch 6/150
1665/1665 [==============================] - 7s 4ms/step - loss: 90.0050 - val_loss: 78.2965
Epoch 7/150
1665/1665 [==============================] - 7s 4ms/step - loss: 84.9528 - val_loss: 66.6397
Epoch 8/150
1665/1665 [==============================] - 7s 4ms/step - loss: 77.6245 - val_loss: 60.9388
Epoch 9/150
1665/1665 [==============================] - 7s 4ms/step - loss: 73.1044 - val_loss: 61.5609

Epoch 78/150
1665/1665 [==============================] - 7s 4ms/step - loss: 16.1864 - val_loss: 65.4834
Epoch 79/150
1665/1665 [==============================] - 7s 4ms/step - loss: 15.7776 - val_loss: 64.4385
Epoch 80/150
1665/1665 [==============================] - 7s 4ms/step - loss: 15.3650 - val_loss: 61.6860
Epoch 81/150
1665/1665 [==============================] - 7s 4ms/step - loss: 15.2137 - val_loss: 61.1428
Epoch 82/150
1665/1665 [==============================] - 7s 4ms/step - loss: 17.6387 - val_loss: 54.5457
Epoch 83/150
1665/1665 [==============================] - 7s 4ms/step - loss: 16.9890 - val_loss: 56.8173
Epoch 84/150
1665/1665 [==============================] - 7s 4ms/step - loss: 16.0390 - val_loss: 54.8291
Epoch 85/150
1665/1665 [==============================] - 7s 4ms/step - loss: 16.6261 - val_loss: 59.1771
Epoch 86/150
1665/1665 [==============================] - 7s 4ms/step - loss: 14.3098 - val_loss: 60.4772
Epoch 87/150
1665/1665 [======================

In [19]:
#Add a global test set (for overall performances)
test_smiles_global = np.concatenate([t[0] for t in test_sets])
test_adducts_global = np.concatenate([t[1] for t in test_sets])
test_ccs_global = np.concatenate([t[2] for t in test_sets])
test_sets.append([test_smiles_global, test_adducts_global, test_ccs_global])
test_sets_names = test_sets_names + ["global"]

In [20]:
deepCCS_model.model.load_weights(model_file)
for i, t in enumerate(test_sets):
    predictions = deepCCS_model.predict(t[0], t[1]).flatten()
    print(test_sets_names[i])
    print("R2: {}".format(r2_score(y_true=t[2], y_pred=predictions)))
    print("Mean relative: {}".format(relative_mean(Y_true=t[2], Y_pred=predictions)))
    print("Median relative: {}".format(relative_median(Y_true=t[2], Y_pred=predictions)))
    print("-------------------------------------")

Agilent_pos
R2: 0.928828206868
Mean relative: 2.88016945287
Median relative: 2.43481952793
-------------------------------------
Agilent_neg
R2: 0.94075991634
Mean relative: 3.34343517894
Median relative: 2.3719368314
-------------------------------------
Waters_pos
R2: 0.878374546753
Mean relative: 5.11035448525
Median relative: 4.27127925365
-------------------------------------
Waters_neg
R2: 0.945657208159
Mean relative: 3.54683678566
Median relative: 2.89346175203
-------------------------------------
PNL
R2: 0.950302081828
Mean relative: 3.29419433286
Median relative: 2.157990832
-------------------------------------
McLean
R2: 0.985577988559
Mean relative: 2.42848105015
Median relative: 1.50084746117
-------------------------------------
CBM
R2: 0.912018040836
Mean relative: 3.0431315599
Median relative: 2.7617935133
-------------------------------------
global
R2: 0.967663890223
Mean relative: 3.45987565868
Median relative: 2.55678369244
-------------------------------------
